In [ ]:
import pandas as pd
df = pd.read_csv('imdb_indonesian.csv')
df = df.drop(columns=['judul_film'])

In [17]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


Karena label kita berupa data kategorikal, maka kita perlu melakukan proses one-hot-encoding.

In [ ]:
# melakukan one-hot-encoding

category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [ ]:
# mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values

In [ ]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [ ]:
# bagi data untuk training dan data untuk testing

from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [ ]:
# ubah setiap kata pada dataset ke dalam bilangan numerik dengan fungsi Tokenizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [ ]:
# arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16,
# serta dimensi dari input sebesar nilai num_words pada objek tokenizer

import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# melatih model

num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 33s - loss: 1.6100 - accuracy: 0.2040 - val_loss: 1.6107 - val_accuracy: 0.2289 - 33s/epoch - 1s/step
Epoch 2/30
26/26 - 29s - loss: 1.6023 - accuracy: 0.2164 - val_loss: 1.6121 - val_accuracy: 0.2040 - 29s/epoch - 1s/step
Epoch 3/30
26/26 - 29s - loss: 1.5472 - accuracy: 0.3159 - val_loss: 1.6314 - val_accuracy: 0.2090 - 29s/epoch - 1s/step
Epoch 4/30
26/26 - 29s - loss: 1.3394 - accuracy: 0.3993 - val_loss: 1.6931 - val_accuracy: 0.2040 - 29s/epoch - 1s/step
Epoch 5/30
26/26 - 30s - loss: 1.1136 - accuracy: 0.4453 - val_loss: 1.9226 - val_accuracy: 0.2388 - 30s/epoch - 1s/step
Epoch 6/30
26/26 - 29s - loss: 0.8996 - accuracy: 0.5659 - val_loss: 2.2580 - val_accuracy: 0.2289 - 29s/epoch - 1s/step
Epoch 7/30
26/26 - 29s - loss: 0.7074 - accuracy: 0.6741 - val_loss: 3.5842 - val_accuracy: 0.2189 - 29s/epoch - 1s/step
Epoch 8/30
26/26 - 29s - loss: 0.5692 - accuracy: 0.7512 - val_loss: 3.4322 - val_accuracy: 0.2139 - 29s/epoch - 1s/step
Epoch 9/30
26/26 - 29s - loss: 0